<a href="https://colab.research.google.com/github/LamuneGitHub/A001_Python_Test/blob/main/EDA_001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

----
----
# # EDA ( Exploratory Data Analysis )( 탐색적 데이터 분석)
## 정의
> : 탐색적 데이터 분석  
: 데이터의 특징을 파악하는것

## 목적
> : 본격적인 데이터 분석이나 모델링 전에 데이터의 특징을 파악하여 분석 계획을 세우기 위한 기초공사 작업  
  : 유의미한 business insight(업무 통찰)을 얻기 위함  
  : 시행착오 비용 최소화

## 과정 및 방법
* Data Description 확인 (데이터 명세서 )  
  : 데이터 명세서를 읽어서 데이터의 특성을 이해 

* 시각적 분석
  * 엑셀등을 이용하여 분석

* 프로그램적 분석
  * info()
  * shape
  * description()
  * isnull().sum()
  * duplicated()
  * 기타 프로그램코딩 활용

> 목표 (프로그램적 분석의)
  * 결측치 확인
    * 컬럼별 결측치 비율
  * 카테고리 값의 비율
  * 가능 범위를 벗어난 값 확인
  * 계산값이 틀린 값 확인
  * 이상한 문자가 들어간 값 확인
  * 중복값

* Feature Engineering (변수 가공)
* Business Insight (업무 통찰)



# ##과정 및 방법

## 기초 셋팅

In [ ]:
# 구글 드라이브 mount
from google.colab import drive
drive.mount('/content/drive')

#import
import re
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# 엑셀 파일 로딩
df_001 = pd.read_excel("https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/stocks/Travel.xlsx", sheet_name = "008770 ")
# CSV 파일 로딩
df_red = pd.read_csv( "https://raw.githubusercontent.com/aniruddhachoudhury/Red-Wine-Quality/master/winequality-red.csv" , sep = ",")
df_white = pd.read_csv( "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv" , sep = ";")
df_movie = pd.read_csv('https://raw.githubusercontent.com/LamuneGitHub/A001_Python_Test/main/data/movie.csv')
df_titanic = sns.load_dataset("titanic")
df_patients = pd.read_csv('https://raw.githubusercontent.com/LamuneGitHub/A001_Python_Test/main/data/patients_info.csv')
df_insuline_test = pd.read_csv('https://raw.githubusercontent.com/LamuneGitHub/A001_Python_Test/main/data/insuline_test.csv')
df_side_effects = pd.read_csv('https://raw.githubusercontent.com/LamuneGitHub/A001_Python_Test/main/data/side_effects.csv')


##### !Tip

* code 뒤에 ; 를 써주면 colab의 결과창 표시가 간단한 버전으로 깔끔하게 나온다. 

## 시각적 탐색


In [ ]:
#pandas에서 DataFrame을 요약해서 표시하지 않도록 설정
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
#맨위 3개
df_titanic.head(3)

In [ ]:
#마지막 3개
df_titanic.tail(3)

In [ ]:
#무작위 3개
df_titanic.sample(3)


## 프로그램적 분석


In [ ]:
# 사망자그룹의 비율 계산
# value_counts(normalize=True) => 비율 
df_temp = df_titanic['survived'].value_counts(normalize=True)
df_temp

In [ ]:
# 소숫점 2째자리까지 반올림
print (df_temp[0].round(2))

In [ ]:
# 사망자 그룹의 숫자 확인
# value_counts()
df_temp = df_titanic['survived'].value_counts() # 갯수 구하기
df_temp

## Feature Engineering


In [ ]:
## 범위에 따라 단계 분류한 값을 age_class 컬럼을 생성하여 입력 
df_temp = df_titanic

bins = [-2, 0, 19, 59, 999]
labels = ['unknown','young', 'middle', 'old']
df_temp['age_class'] = pd.cut(x=df_temp['age'], bins=bins, labels=labels)
df_temp.head(3)

In [ ]:
# 테이블별 중복 column name 확인
all_columns = pd.Series(list(df_patients) + list(df_insuline_test) + list(df_side_effects))
all_columns
all_columns.duplicated() # 중복여부 배열 
all_columns[all_columns.duplicated()]

## 시각화


In [ ]:
# 생존여부, 성별 별 평균요금 df 조회
df_temp = df_titanic.groupby( ['survived' ,'sex'] ).mean()
df_temp

In [ ]:
# 막대그래프 표시
df_temp.plot( kind = 'bar')

----
----
# # Data Wrangling (데이터 정제) , Data Preprocessing (데이터 전처리)  
  : 실제 작업의 70~80%가 소요 된다고 함 

  * 데이터 품질
    * 결측값 처리
    * 잘못된 값 처리
  * 구조적 품질
    * 정규화



In [ ]:
# DataFrame 복사
df_patients_clean = df_patients.copy()

### Missing value (결측값)

In [ ]:
# 환자 데이터 중 키, 몸무게, 체질량지수 컬럼의 결측값 확인
df_patients_error_tmp = df_patients[df_patients['키'].isnull() | df_patients['몸무게'].isnull() | df_patients['체질량지수'].isnull()]
df_patients_error_tmp.head(3)

### 잘못된 값 

In [ ]:
reg_con_01 = r'[_,!,.,@#$%()^\d+]'

# 특수 문자가 포함되었는지 여부 체크
def has_errors(inputString):
  return bool(re.search( reg_con_01 , inputString))


# 이름에 섞여 있는 오류 데이터를 확인
error_condition = df_patients_clean.이름.apply(has_errors)
error_names = df_patients_clean[error_condition]
print(error_names)

print("-------------")
# 코드가 정확하게 오류들을 제거하는지 확인 
error_names.이름 = df_patients_clean.이름.str.replace(reg_con_01, '')
print(error_names)

print("-------------")
# 반영
df_patients_clean.이름 = df_patients_clean.이름.str.replace(reg_con_01, '')

# 반영결과 확인
df_patients_clean[error_condition] 

### 컬럼의 분리

In [ ]:
# 환자나이 컬럼 분리
# 나이 컬럼에서 숫자만 뽑아내어 int로 변환 후 환자나이 컬럼을 생성 
df_patients_clean['환자나이'] = df_patients_clean.나이.str.extract('(\d+)').astype('int') 

# 성별 컬럼 분리
# 나이 컬럼에서 숫자를 모두 제거 후 성별 컬럼을 생성
df_patients_clean['성별'] = df_patients_clean.나이.str.replace(r'[^a-zA-Z]', '', regex=True) #숫자만 삭제 한다.

# 기존 나이 컬럼 제거
patients_clean = df_patients_clean.drop('나이', axis='columns' )


In [ ]:
# 성별을 category 데이터 타입으로 변화 , 영어를 한글로 변환
patients_clean.성별 = patients_clean.성별.replace({'male':'남', 'female':'여'})
patients_clean.성별 = patients_clean.성별.astype('category')

## 구조적 문제
df1.melt(id_vars='index', value_vars=['A', 'B'])

pd.melt(df1, id_vars = 'index', var_name='variable', value_name='value')